In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

xx = []
y = []
for k in range(10,100):
    xx.append((k,100))
    y.append((k,10))
xx.extend(y)
del y
rvals = [ a[0]/a[1] for a in xx ]
del xx

def id2el(Id):
    """Constructs the graph associated to the number Id.
    """
    el = []

    Id = int(Id)

    mask = int(1)
    u = int(0)
    v = int(1)
    while u < 10:
        if (Id & mask):
            el.append((u,v))
    
        mask = (mask << int(1))
        v += int(1)
    
        if v > 10:
            u += 1
            v = u+1
    return el
id2G  = lambda Id : nx.from_edgelist (id2el(np.uint64(Id)))


In [2]:
colorcycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
X = pd.read_csv("evaluacion.txt", header=None)
X.columns = ("ID", "kind", *rvals)

In [4]:
kinds = [k for k in set(X["kind"])]
IDs = list(set( x for x in X["ID"] if x not in [17450410000, 17586207775, 17449355267]))
IDs = [17586207775, 17449355267, 17450410000] + IDs

In [5]:
for k in kinds:
    TMP = X[X["kind"] == k]
    TMP.pop("kind")
    
    TMP.index = TMP.pop("ID")
    
    globals()[k.strip()] =  TMP.transpose()

In [8]:
for name in ["conditional", "unconditional"]:
    fig,axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4), sharey=True, sharex=True)
    figdiff,ax = plt.subplots(1,1)
    for ID in IDs:
        G = id2G(ID)
        style = {"c":"k", "alpha":0.1}
        back = False
        if ID == 17450410000:
            style = {"c":colorcycle[1], "zorder": 100, "label": "$K_{1,5}$"}
        elif ID == 17586207775:
            style = {"c":colorcycle[0], "zorder": 100, "label": "$K_6$"}
        elif ID == 17449355267:
            style = {"c":colorcycle[2], "zorder": 100, "label": "$C_6$"}
        elif nx.is_tree(G):
            style = {"c":colorcycle[3], "alpha":.6, "zorder": 10}
            back = True
        elif len(set(G.degree(u) for u in G)) == 1:
            style = {"c":colorcycle[4], "alpha":.75, "zorder": 20}
            back = True
        
        m = globals()[f"moran_{name}_mean_time_from1"][ID].values
        b = globals()[f"bernoulli_{name}_mean_time_from1"][ID].values
        axs[0].plot(rvals, m, **style)
        axs[1].plot(rvals, b, **style)
        if back:
            axs[0].plot(rvals, m, c="white", zorder=2, alpha=.5)
            axs[1].plot(rvals, b, c="white", zorder=2, alpha=.5)

        ax.plot(rvals, m/b, **style)
        ax.plot([rvals[0], rvals[-1]], [1,1], ":", c="gray", linewidth=1)
        
    axs[0].set_xscale("log")
    fig.suptitle(f"{name.title()} mean fixation times")
    axs[0].set_title("Moran")
    axs[1].set_title("Bernouilli")
    axs[0].set_xlabel("Fitness")
    axs[1].set_xlabel("Fitness")
    axs[0].set_ylabel("Time")
    fig.tight_layout()
    
    ax.set_xscale("log")
    if name == "conditional":
        ax.set_ylabel('$T_1^M/T^b_1$')
    else:
        ax.set_ylabel(r'$\tau_1^M/\tau^b_1$')
    ax.set_xlabel("Fitness")

    axs[1].plot([], **{"c":colorcycle[3], "alpha":.5, "zorder": 1}, label="Trees")
    axs[1].plot([], **{"c":colorcycle[4], "alpha":.75, "zorder": 2}, label="Isothermal")
    axs[1].legend()
    fig.savefig(f"{name}.pdf")
    figdiff.savefig(f"{name}_quotient.pdf")
    ax.set_yscale("log")
    figdiff.savefig(f"{name}_quotient_log.pdf")
    plt.close("all")
    